# TEST1: add non-disaster 400 lines
## performing measurement
### Disaster Tweet Classification with Disaster Tweets

------------------
### GOAL
- comparing the performance of different word representation
    - word2vec vs. word count
- Predicting whether a given tweet is about a real disaster or not
- if so, predict a `1`. if not, predict a `0`


### Reference
- [competition main page](https://www.kaggle.com/c/nlp-getting-started)
- [word2vec code](https://www.kaggle.com/slatawa/simple-implementation-of-word2vec)
- [word2vec-implementation-for-beginner](https://www.kaggle.com/manavkapadnis/nlp-word2vec-implementation-for-beginner)
- [word count](https://www.kaggle.com/datarohitingole/disaster-tweet-classification-ridgeclassifiercv)
- [comparing the performance of different Machine Learning Algorithm](https://dibyendudeb.com/comparing-machine-learning-algorithms/)
- [Text Classification with NLP: Tf-Idf vs Word2Vec vs BERT](https://towardsdatascience.com/text-classification-with-nlp-tf-idf-vs-word2vec-vs-bert-41ff868d1794)

# 0. Importing Libraries

In [1]:
# for loading and preprocessing the data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re

# for training the model
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import tree, linear_model, neighbors, naive_bayes, ensemble
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# for evaluating classification model
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

# for data cleaning
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
nltk.download('punkt')


# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM

# for word2vec
import gensim
from gensim.models import Word2Vec

# Comparing all machine learning algorithms
from sklearn.metrics import mean_squared_error,confusion_matrix, precision_score, recall_score, auc,roc_curve

[nltk_data] Downloading package wordnet to /Users/mac/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 2. Data Preprocessing
## Contents
### 1. Clean the data
- dealing with missing values
- replace some commonly occuring shorthands
- remove any characters other then alphabets
- convert all dicitonary to lower case(for consistency)
- lemmatize

### 2-1. word tokenization and word2vec

### 2-2. Convert text to vectors using Counter vectorizer





## Data Description
--------------
### Files
- `train.csv` : the training set
- `test.csv` : the test set
- `sample_submission.csv` : a sample submission file in the correct format

### Columns
- `id` : a unique identifier for each tweet
- `text` : the text of the tweet
- `location` : the location the tweet was sent from 
- `keyword` : a particular keyword from th tweet
- `target` : in train.csv only, this denotes whether a tweet is about a real disaster(1) or not(0)

In [2]:
# loading the data set
data_path = './data/'
add_nonD = pd.read_csv(data_path + 'train_add_nonD400.csv')

In [3]:
print('train_shape:', add_nonD.shape)
add_nonD.head()

train_shape: (8013, 5)


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
# split the data <train : test = 8 : 2>
train, test = train_test_split(add_nonD, test_size = 0.20, random_state = 0)

In [5]:
print('train shape:', train.shape)
train.head()

train shape: (6410, 5)


,id,keyword,location,text,target
7233,10359,weapons,california | oregon | peru |,@DorisMatsui thank you for supporting the Pres...,0
4685,6661,landslide,New York City,Inbounds/ Out of Bounds:\n\nWhile many picked ...,1
1722,2485,collided,bk.,She looked back &amp; her daughter &amp; said ...,0
3433,4909,explode,New Hampshire,@DelDryden If I press on the twitch will my he...,0
1595,2303,cliff%20fall,The Netherlands,#NowPlaying * Cliff Richard - I Could Easily F...,0


In [6]:
print('test shape:', test.shape)
test.head()

test shape: (1603, 5)


,id,keyword,location,text,target
1060,1531,bomb,NaN,beforeitsnews : Global Derivatives: $1.5 Quadr...,0
5463,7793,police,Los Angeles,.@slosheriff: 2 South Gate police officers and...,0
5430,7749,police,NaN,These fucking police can't touch me these fuc...,0
776,1123,blew%20up,NaN,Ye did the same thing to Big Sean and he still...,0
1845,2655,crush,Kaneohe,@kuualohax more like you love your husband but...,0


## 1. Clean the data
#### (1) Dealing with Missing Values

In [7]:
all_data = [train,test]
for data in all_data:
    data.drop(["location", "id"], axis = 1, inplace = True)

/Users/mac/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [8]:
# data prepocessing with regrex

def remove_URL(text): # remove url pattern in text
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

def remove_html(text): # remove html pattern in text
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return html.sub(r'', text)
    #return re.sub(html, '', text)

def remove_punct(text): # remove punctuation in text: (;, ', ", :, ., , etc..)
  table = str.maketrans('', '', string.punctuation)
  return text.translate(table)

#### (2) Replace some commonly occuring shorthands

In [9]:
for data in all_data:
  data['text'] = data['text'].apply(lambda x: remove_URL(x))
  data['text'] = data['text'].apply(lambda x: remove_html(x))
  data['text'] = data['text'].apply(lambda x: remove_punct(x))

<ipython-input-9-650a1bb61e39>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: remove_URL(x))
<ipython-input-9-650a1bb61e39>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: remove_html(x))
<ipython-input-9-650a1bb61e39>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [10]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"you'll", "you will", text)
    text = re.sub(r"i'll", "i will", text)
    text = re.sub(r"she'll", "she will", text)
    text = re.sub(r"he'll", "he will", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"there's", "there is", text)
    text = re.sub(r"here's", "here is", text)
    text = re.sub(r"who's", "who is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"shouldn't", "should not", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"   ", " ", text) # Remove any extra spaces
    return text

In [11]:
train['clean_text'] = train['text'].apply(clean_text)
test['clean_text'] = test['text'].apply(clean_text)

<ipython-input-11-fcb0c48fb32b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['clean_text'] = train['text'].apply(clean_text)
<ipython-input-11-fcb0c48fb32b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['clean_text'] = test['text'].apply(clean_text)


In [12]:
train.head()

,keyword,text,target,clean_text
7233,weapons,DorisMatsui thank you for supporting the Presi...,0,dorismatsui thank you for supporting the presi...
4685,landslide,Inbounds Out of Bounds\n\nWhile many picked th...,1,inbounds out of bounds\n\nwhile many picked th...
1722,collided,She looked back her daughter said everyone l...,0,she looked back her daughter said everyone l...
3433,explode,DelDryden If I press on the twitch will my hea...,0,deldryden if i press on the twitch will my hea...
1595,cliff%20fall,NowPlaying Cliff Richard I Could Easily Fall...,0,nowplaying cliff richard i could easily fall...


#### (3) remove any characters other then alphabets
#### (4) convert all dicitonary to lower case(for consistency)
#### (5) lemmatize

In [13]:
def massage_text(text):
    ## remove anything other then characters and put everything in lowercase
    tweet = re.sub("[^a-zA-Z]", ' ', text) # (3)
    tweet = tweet.lower() # (4) 
    tweet = tweet.split()

    lem = WordNetLemmatizer() # (5)
    tweet = [lem.lemmatize(word) for word in tweet
             if word not in set(stopwords.words('english'))]
    tweet = ' '.join(tweet)
    return tweet
    print('--here goes nothing')
    print(text)
    print(tweet)

train['clean_text'] = train['text'].apply(massage_text)
test['clean_text'] = test['text'].apply(massage_text)

<ipython-input-13-63aca0e435f3>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['clean_text'] = train['text'].apply(massage_text)
<ipython-input-13-63aca0e435f3>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['clean_text'] = test['text'].apply(massage_text)


In [14]:
train.iloc[0:10][['text','clean_text']]

,text,clean_text
7233,DorisMatsui thank you for supporting the Presi...,dorismatsui thank supporting president irandea...
4685,Inbounds Out of Bounds\n\nWhile many picked th...,inbounds bound many picked nats win nl east la...
1722,She looked back her daughter said everyone l...,looked back daughter said everyone loved pictu...
3433,DelDryden If I press on the twitch will my hea...,deldryden press twitch head explode
1595,NowPlaying Cliff Richard I Could Easily Fall...,nowplaying cliff richard could easily fall lov...
3010,Let it be gone away like a dust in the wind B...,let gone away like dust wind big wind like tor...
1001,Im not a Drake fan but I enjoy seeing him body...,im drake fan enjoy seeing bodybagging people g...
6100,AP \n Too slow report the sinking boat in the ...,ap slow report sinking boat mediterranean sea ...
5244,Refugio oil spill may have been costlier bigge...,refugio oil spill may costlier bigger projecte...
3719,When the world say finish God says dont fear ...,world say finish god say dont fear choosegod r...


## 2-1. Word Tokenization and word2vec
- tokenize the clean text column

In [15]:
train['tokens'] = train['clean_text'].apply(lambda x: word_tokenize(x))
test['tokens'] = test['clean_text'].apply(lambda x: word_tokenize(x))

<ipython-input-15-96b940e3b626>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['tokens'] = train['clean_text'].apply(lambda x: word_tokenize(x))
<ipython-input-15-96b940e3b626>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['tokens'] = test['clean_text'].apply(lambda x: word_tokenize(x))


In [16]:
train.head()

,keyword,text,target,clean_text,tokens
7233,weapons,DorisMatsui thank you for supporting the Presi...,0,dorismatsui thank supporting president irandea...,"[dorismatsui, thank, supporting, president, ir..."
4685,landslide,Inbounds Out of Bounds\n\nWhile many picked th...,1,inbounds bound many picked nats win nl east la...,"[inbounds, bound, many, picked, nats, win, nl,..."
1722,collided,She looked back her daughter said everyone l...,0,looked back daughter said everyone loved pictu...,"[looked, back, daughter, said, everyone, loved..."
3433,explode,DelDryden If I press on the twitch will my hea...,0,deldryden press twitch head explode,"[deldryden, press, twitch, head, explode]"
1595,cliff%20fall,NowPlaying Cliff Richard I Could Easily Fall...,0,nowplaying cliff richard could easily fall lov...,"[nowplaying, cliff, richard, could, easily, fa..."


#### convert our data(words) into vectors

In [17]:
#first, create a list corpus which we would be using to train word2vec mappings
def fn_pre_process_data(doc):
    for rec in doc:
        yield gensim.utils.simple_preprocess(rec)

corpus = list(fn_pre_process_data(train['clean_text']))
corpus += list(fn_pre_process_data(test['clean_text']))

In [18]:
#inititate the embedding model, we will come back to the passed arguments later
print('initiated ...')
wv_model = Word2Vec(corpus,vector_size=150,window=3,min_count=2)
#wv_model.build_vocab(corpus)
wv_model.train(corpus,total_examples=len(corpus),epochs=10)
#wv_model.save(data_path + 'word2vec.model')

initiated ...


(635899, 746280)

In [19]:
# convert the train and text tokens
def get_word_embeddings(token_list,vector,k=150):
    if len(token_list) < 1:
        return np.zeros(k)
    else:

        vectorized = [vector.wv[word] if word in vector.wv else np.random.rand(k) for word in token_list] 
    
    sum = np.sum(vectorized,axis=0)
    ## return the average
    return sum/len(vectorized)       

def get_embeddings(tokens,vector):
        embeddings = tokens.apply(lambda x: get_word_embeddings(x, wv_model))
        return list(embeddings)

In [20]:
train_embeddings = get_embeddings(train['tokens'],wv_model)
test_embeddings = get_embeddings(test['tokens'],wv_model)

In [21]:
model_path = './model/'
wv_model.save(model_path + 'word2vec_1.model')

## 2-2. Convert text to vectors using Counter vectorizer

### What is the Count Vectorizer?
- convert a collection of text documents to a matrix of token counts

### How to Use
```python
# python example code
corpus = ["This is the first document", "This document is the second document", "And this is the thrid one"]
vectorize = CounterVectorize()
X = vectorize.fit_transform(corpus)
```
- vectorizer.get_feature_names_out()
> array(['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this'], ...)
- X.toarray()
> [[0 1 1 1 0 0 1 0 1]  
 [0 2 0 1 0 1 1 0 1]  
 [1 0 0 1 1 0 1 1 1]]

In [22]:
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(max_features=2000)

X = count_vectorizer.fit_transform(train["clean_text"]).toarray()
test_tmp = count_vectorizer.transform(test["clean_text"]).toarray()
y = train['target']

In [23]:
X_train = X
X_test = test_tmp
y_train = train['target']
y_test = test['target']

# 3. Model
## Contents
- train the model
    - RidgeClassifierCV
    - sgd classifier
    - BernoulliNB 
    - RandomForest

## Model Description
--------------
### Ensemble
- Combine the predictions of several base estimators built with a given learning algorithm 
    - in order to improve generalizability / robustness over a single estimator.
- Boosting of Ensemble types

### Performance - f1-score


## 3-2. Train the model

In [24]:
MLA = [
    #Ensemble Methods
    ensemble.RandomForestClassifier(),
    
    #GLM
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB()
    ]

# Comapring all MLA
## word2vec
- precision
- recall
- accuracy
- f1-score

In [25]:
train_embeddings = np.array(train_embeddings)
test_embeddings = np.array(test_embeddings)

In [26]:
# Comparing all machine learning algorithms
from sklearn.metrics import mean_squared_error,confusion_matrix, precision_score, recall_score, auc,roc_curve

row_index = 0
MLA_columns = []
MLA_compare = pd.DataFrame(columns = MLA_columns)

for alg in MLA:
    predicted = alg.fit(train_embeddings, y_train).predict(test_embeddings)
    fp, tp, th = roc_curve(y_test, predicted)

    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index,'MLA used'] = MLA_name
    MLA_compare.loc[row_index, 'Train Accuracy'] = round(alg.score(train_embeddings,y_train), 4)
    MLA_compare.loc[row_index, 'Test Accuracy'] = round(alg.score(test_embeddings, y_test), 4)
    
    
    recall = recall_score(test['target'], predicted)
    precision = precision_score(test['target'], predicted)
    MLA_compare.loc[row_index, 'Precission'] = precision
    MLA_compare.loc[row_index, 'Recall'] = recall
    MLA_compare.loc[row_index, 'F1-score'] = round((2*precision*recall)/(precision+recall),4)
    MLA_compare.loc[row_index, 'AUC'] = auc(fp, tp)

    row_index+=1

In [27]:
# f1-score 기준 정렬
MLA_compare.sort_values(by = ['F1-score'], ascending = False, inplace = True)    
MLA_compare

,MLA used,Train Accuracy,Test Accuracy,Precission,Recall,F1-score,AUC
0,RandomForestClassifier,0.9906,0.7386,0.797701,0.511799,0.6235,0.708332
2,SGDClassifier,0.7495,0.7037,0.703407,0.517699,0.5964,0.678850
3,BernoulliNB,0.6031,0.6057,0.529040,0.617994,0.5701,0.607375
1,RidgeClassifierCV,0.7417,0.7124,0.798898,0.427729,0.5572,0.674405


# Comapring all MLA
## wordCount
- precision
- recall
- accuracy
- f1-score

In [28]:
# Comparing all machine learning algorithms
from sklearn.metrics import mean_squared_error,confusion_matrix, precision_score, recall_score, auc,roc_curve

row_index = 0
MLA_columns = []
MLA_compare = pd.DataFrame(columns = MLA_columns)

for alg in MLA:
    predicted = alg.fit(X_train, y_train).predict(X_test)
    fp, tp, th = roc_curve(y_test, predicted)

    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index,'MLA used'] = MLA_name
    MLA_compare.loc[row_index, 'Train Accuracy'] = round(alg.score(X_train, y_train), 4)
    MLA_compare.loc[row_index, 'Test Accuracy'] = round(alg.score(X_test, y_test), 4)
    
    
    recall = recall_score(y_test, predicted)
    precision = precision_score(y_test, predicted)
    MLA_compare.loc[row_index, 'Precission'] = precision
    MLA_compare.loc[row_index, 'Recall'] = recall
    MLA_compare.loc[row_index, 'F1-score'] = round((2*precision*recall)/(precision+recall),4)
    MLA_compare.loc[row_index, 'AUC'] = auc(fp, tp)

    row_index+=1

In [29]:
# f1-score 기준 정렬
MLA_compare.sort_values(by = ['F1-score'], ascending = False, inplace = True)    
MLA_compare

,MLA used,Train Accuracy,Test Accuracy,Precission,Recall,F1-score,AUC
1,RidgeClassifierCV,0.8786,0.8029,0.832721,0.668142,0.7414,0.784882
3,BernoulliNB,0.8526,0.7998,0.821622,0.672566,0.7397,0.782770
0,RandomForestClassifier,0.9824,0.7817,0.761146,0.705015,0.7320,0.771426
2,SGDClassifier,0.9207,0.7785,0.765189,0.687316,0.7242,0.766361
